# Micro-haplotype assembly in an autopolyploid bi-parental cross

*(Last updated for MCHap version 0.9.3)*

This notebook demonstrates micro-haplotype assembly and genotype re-calling in a small bi-parental cross.
It introduces the following topics:

- **De novo haplotype assembly with `mchap assemble`**
  - Input files
  - Basic assembly
  - Reporting posterior allele frequencies
  - Adjusting the threshold for reporting alleles
- **Re-calling genotypes with `mchap call`** (two-step approach)
  - Basic re-calling
  - Specifying prior allele frequencies
  - Two-step approach with a pooled assembly

**Software requirements:**

This notebook uses the [bash-kernel](https://github.com/takluyver/bash_kernel) for Jupyter which can be installed with `pip install bash_kernel`.
Alternatively, the code from this notebook may be run in a unix-like bash environment.

In addition to using the MCHap software, these examples also require the `bgzip` and `tabix` tools which are part of [`htslib`](https://github.com/samtools/htslib).

**Data sources:**

The bi-parental population used within this notebook is a small subset of the population published by:
J Tahir, C Brendolise, S Hoyte, M Lucas, S Thomson, K Hoeata, C McKenzie, A Wotton, K Funnell, E Morgan, D Hedderley, D Chagné, P M Bourke, J McCallum, S E Gardiner,* and L Gea 
“QTL Mapping for Resistance to Cankers Induced by Pseudomonas syringae pv. actinidiae (Psa) in a Tetraploid Actinidia chinensis Kiwifruit Population” 
Pathogens 2020, 9, 967; doi:10.3390/pathogens9110967

- Raw sequences: [10.5281/zenodo.4285665](https://zenodo.org/record/4285666) and [10.5281/zenodo.4287636](https://zenodo.org/record/4287637)
- Reference genome: DOI [10.5281/zenodo.5717386](https://zenodo.org/record/5717387) (chromosome 1 only)


## De novo haplotype assembly with `mchap assemble`

*For more background information on* `mchap assemble` *see the online [documentation](https://github.com/PlantandFoodResearch/MCHap/blob/master/docs/assemble.rst).*

### Input files

The required input files have been organised by file type:

- `input/bam` BAM alignment files for the parents and progeny samples
- `input/bed` Target loci for haplotype assembly
- `input/vcf` VCF file of "basis" SNVs
- `input/fasta`: Reference genome

In this example we use one BAM file per sample:

In [1]:
ls input/bam

parent1.loci.bam         progeny006.loci.bam.bai  progeny014.loci.bam
parent1.loci.bam.bai     progeny007.loci.bam      progeny014.loci.bam.bai
parent2.loci.bam         progeny007.loci.bam.bai  progeny015.loci.bam
parent2.loci.bam.bai     progeny008.loci.bam      progeny015.loci.bam.bai
progeny001.loci.bam      progeny008.loci.bam.bai  progeny016.loci.bam
progeny001.loci.bam.bai  progeny009.loci.bam      progeny016.loci.bam.bai
progeny002.loci.bam      progeny009.loci.bam.bai  progeny017.loci.bam
progeny002.loci.bam.bai  progeny010.loci.bam      progeny017.loci.bam.bai
progeny003.loci.bam      progeny010.loci.bam.bai  progeny018.loci.bam
progeny003.loci.bam.bai  progeny011.loci.bam      progeny018.loci.bam.bai
progeny004.loci.bam      progeny011.loci.bam.bai  progeny019.loci.bam
progeny004.loci.bam.bai  progeny012.loci.bam      progeny019.loci.bam.bai
progeny005.loci.bam      progeny012.loci.bam.bai  progeny020.loci.bam
progeny005.loci.bam.bai  progeny013.loci.bam      progeny020.loci.

The BED file specifies the genomic coordinates of assembly targets. The fourth column (loci ID) of the BED file is optional but, if present, the loci IDs will be included in the output VCF file:

In [2]:
cat input/bed/targets20.bed

chr1	17590	17709	locus001
chr1	89897	90016	locus002
chr1	107231	107350	locus003
chr1	267278	267397	locus004
chr1	272035	272154	locus005
chr1	377108	377223	locus006
chr1	403018	403137	locus007
chr1	463972	464091	locus008
chr1	530671	530790	locus009
chr1	532849	532968	locus010
chr1	538202	538321	locus011
chr1	568848	568967	locus012
chr1	585411	585530	locus013
chr1	596680	596799	locus014
chr1	603851	603970	locus015
chr1	684808	684927	locus016
chr1	695531	695650	locus017
chr1	717620	717739	locus018
chr1	809104	809223	locus019
chr1	1027533	1027652	locus020


Note that the examples in this notebook only require a subset of these loci as defined in:

In [3]:
cat input/bed/targets4.bed

chr1	17590	17709	locus001
chr1	568848	568967	locus012
chr1	684808	684927	locus016
chr1	809104	809223	locus019


The input VCF file is used to specify basis SNVs for haplotype alleles. These may be multi-allelic SNVs. Any sample data within this VCF file will be ignored:

In [4]:
head -n 20 input/vcf/snvs.vcf

##fileformat=VCFv4.3
##fileDate=20221212
##source=NotPretty
##reference=chr1.fa
##contig=<ID=chr1,length=21898217,assembly=Red5v2,species="Actinidia chinensis">
##phasing=partial
#CHROM	POS	ID	REF	ALT	QUAL	FILTER	INFO
chr1	17495	.	A	C	.	.	.
chr1	17507	.	T	C	.	.	.
chr1	17528	.	T	A	.	.	.
chr1	17564	.	A	G	.	.	.
chr1	17576	.	T	C	.	.	.
chr1	17587	.	C	T	.	.	.
chr1	17591	.	G	A	.	.	.
chr1	17602	.	G	A	.	.	.
chr1	17612	.	A	G	.	.	.
chr1	17616	.	A	G	.	.	.
chr1	17624	.	C	T	.	.	.
chr1	17625	.	G	C	.	.	.
chr1	17629	.	G	T	.	.	.


### Basic assembly

A simple `mchap assemble` command:

In [5]:
mchap assemble \
        --bam input/bam/*.bam \
        --targets input/bed/targets4.bed \
        --variants input/vcf/snvs.vcf.gz \
        --reference input/fasta/chr1.fa.gz \
        --ploidy 4 \
        --inbreeding 0.01 \
        | bgzip > assemble.vcf.gz
tabix -p vcf assemble.vcf.gz

Notes:
- The `--bam` argument may be file with a list of BAM paths ([documentation](https://github.com/PlantandFoodResearch/MCHap/blob/master/docs/assemble.rst#analyzing-many-samples))
- The `--ploidy` and `--inbreeding` values can be specified per sample using a simple tabular file ([documentation](https://github.com/PlantandFoodResearch/MCHap/blob/master/docs/assemble.rst#sample-parameters))
- The `--inbreeding` argument will default to `0` which may be unrealistic and it's often better to guess a "sensible" value ([documentation](https://github.com/PlantandFoodResearch/MCHap/blob/master/docs/assemble.rst#sample-parameters))
- The output of `mchap assemble` is piped into `bgzip` and the resulting compressed VCF file is saved as `assemble.vcf.gz`
- The `tabix` tool is then used to index the compressed VCF file

Look at the VCF header information:

In [6]:
zcat assemble.vcf.gz | grep "^#"

##fileformat=VCFv4.3
##fileDate=20240315
##source=mchap v0.9.3
##phasing=None
##commandline="/home/cfltxm/Software/miniconda3/envs/mchap/bin/mchap assemble --bam input/bam/parent1.loci.bam input/bam/parent2.loci.bam input/bam/progeny001.loci.bam input/bam/progeny002.loci.bam input/bam/progeny003.loci.bam input/bam/progeny004.loci.bam input/bam/progeny005.loci.bam input/bam/progeny006.loci.bam input/bam/progeny007.loci.bam input/bam/progeny008.loci.bam input/bam/progeny009.loci.bam input/bam/progeny010.loci.bam input/bam/progeny011.loci.bam input/bam/progeny012.loci.bam input/bam/progeny013.loci.bam input/bam/progeny014.loci.bam input/bam/progeny015.loci.bam input/bam/progeny016.loci.bam input/bam/progeny017.loci.bam input/bam/progeny018.loci.bam input/bam/progeny019.loci.bam input/bam/progeny020.loci.bam --targets input/bed/targets4.bed --variants input/vcf/snvs.vcf.gz --reference input/fasta/chr1.fa.gz --ploidy 4 --inbreeding 0.01"
##randomseed=42
##contig=<ID=chr1,length=21898217>
##

Look at `locus001`:

In [7]:
zcat assemble.vcf.gz | grep "locus001"

chr1	17591	locus001	GTTATTGGACAGTGACGATGGAGTGATTGCTGGCGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGTACAC	ATTATTGGACAATGACGATGGGGTGGTTGCTGGCCCAGTCCGCCAGCACCACCACCACCAAGTCAACATGTCGGACATTTATGGGGTGGTGCCACGAAACCTGATCACAAATGGCGCAC,GTTATTGGACAGTGACGATGGAGTGATTGCTGGTGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGCACAC,ATTATTGGACAATGACGATGGGGTGGTTGCTGGCCCAGTCCGCCAGCACCACCACCACCAAGTCAACATGTCGGACATTTATGGGGTGGTGCCACGAAACCTGATCACAAATGGTGCAC	.	PASS	AN=88;AC=22,12,0;NS=22;DP=619;RCOUNT=847;END=17709;NVAR=13;SNVPOS=1,12,22,26,34,35,39,65,73,96,104,115,116	GT:GQ:PHQ:DP:RCOUNT:RCALLS:MEC:MECP:GPM:PHPM:MCI	0/0/1/2:28:28:30:43:383:2:0.005:0.998:0.998:0	0/0/0/1:3:3:22:28:280:2:0.007:0.48:0.48:0	0/0/1/2:60:60:31:41:401:1:0.002:1:1:0	0/0/1/2:6:6:14:26:176:0:0:0.752:0.761:0	0/0/1/2:17:18:25:31:320:0:0:0.982:0.983:0	0/0/1/2:60:60:31:46:408:0:0:1:1:0	0/0/0/1:28:28:22:33:291:0:0:0.998:0.998:0	0/0/1/2:27:60:21:26:272:1:0.004:0.998:1:0	0/0/1/2:3:3

It looks like `locus001` assembled fairly well given that all progeny genotypes seem to contain some combination of parental alleles (the first two genotypes, `0/0/1/2` and `0/0/0/1` are the parents.

Note that allele `3` was reported but not called in any samples.
This is because MCHap reports all haplotype-alleles that have a probability of occurring in any sample which is above a certain threshold.
We will see how to adjust this threshold bellow.

Next we'll look at `locus012`:

In [8]:
zcat assemble.vcf.gz | grep "locus012"

chr1	568849	locus012	TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA	TTGATAGGTTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATGGATTGGAAATGAGAGGTGTGAAGGTGCATACTCTCAACATAACACAGTTCTCCGAGTATCGAAAAGACGGTCACCCCTCAATTTACAGAAGGCAATGGGTGGCTTTACCAGAA,TTGATAGATTGGAAATGAGAGGGGTGAATGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGGTTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTGTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATGGATTGGAAATGAGAGGTGTGAAGGTGCATACTCTCAACATAACACAGTTCTCCGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAG

It looks like `locus012` assembled poorly, the symptoms of a poor assembly include:

- Many alleles (13 called, 16 reported)
- Highly heterozygous genotypes
- Some genotypes have unknown alleles (e.g., `1/2/14/.`) resulting from genotype calls with un-reported alleles
- Progeny containing alleles that are not present in their parents
- Low `GQ` and `PHQ` scores (also `GPM` and `PHPM`)
- Several `MCI` values of `2` indicating incongruence between MCMC replicates (putative CNV))

There are two main reasons of bad assemblies:

- Low read depth relative to ploidy, or few reads connecting basis SNVs.
- More alleles present than should be possible given the ploidy (as indicated by an `MCI` of `2`).

There may be several causes of excess alleles:

- Incorrect  ploidy
- Sample contamination
- Of target reads
- Copy number variation

Note that the reference allele was not found as indicated by the `REFMASKED` tag

Finally we'll look at `locus19` in which all samples have a read depth of zero:

In [9]:
zcat assemble.vcf.gz | grep "locus019"

chr1	809105	locus019	GGGTGTGAAAATTGGGCTATTAGTTCTATTTATTATGGGTGCTAGCTGGACTTGTGATGCCAGAGAAGTAATGACCACCAGTCTCTCTAGTGGAGAAACCATGATCTCTGATGTTTCAG	.	.	NOA	AN=0;AC=.;REFMASKED;NS=0;DP=0;RCOUNT=0;END=809223;NVAR=12;SNVPOS=8,9,10,19,44,48,52,62,70,77,85,104	GT:GQ:PHQ:DP:RCOUNT:RCALLS:MEC:MECP:GPM:PHPM:MCI	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:0:.:0.002:0.002:0	./././.:0:0:0:0:0:

Note that genotype calling at this locus has not been "skipped",
but rather the called alleles did not meet the threshold for reporting.
This includes the reference allele hence the `REFMASKED` tag.

### Reporting posterior allele frequencies

We can get MCHap to report posterior allele frequencies using `--report AFP`:

In [10]:
mchap assemble \
        --bam input/bam/*.bam \
        --targets input/bed/targets4.bed \
        --variants input/vcf/snvs.vcf.gz \
        --reference input/fasta/chr1.fa.gz \
        --ploidy 4 \
        --inbreeding 0.01 \
        --report AFP \
        | bgzip > assemble.AFP.vcf.gz
tabix -p vcf assemble.AFP.vcf.gz

The `--report` argument can be used to add optional fields:

- `AFP` posterior allele frequencies (multiply by ploidy to get the posterior mean dosage)
- `GP` genotype posterior distribution
- `GL` genotype log-likelihoods

Note that the posterior distributions will be **truncated** to the *reported haplotype alleles*

(see the [documentation](https://github.com/PlantandFoodResearch/MCHap/blob/master/docs/assemble.rst#output-parameters) for more detail)

`AFP` will be reported for every sample. The population mean `AFP` will also be reported in the `INFO` column:

In [11]:
zcat assemble.AFP.vcf.gz | grep "locus001"

chr1	17591	locus001	GTTATTGGACAGTGACGATGGAGTGATTGCTGGCGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGTACAC	ATTATTGGACAATGACGATGGGGTGGTTGCTGGCCCAGTCCGCCAGCACCACCACCACCAAGTCAACATGTCGGACATTTATGGGGTGGTGCCACGAAACCTGATCACAAATGGCGCAC,GTTATTGGACAGTGACGATGGAGTGATTGCTGGTGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGCACAC,ATTATTGGACAATGACGATGGGGTGGTTGCTGGCCCAGTCCGCCAGCACCACCACCACCAAGTCAACATGTCGGACATTTATGGGGTGGTGCCACGAAACCTGATCACAAATGGTGCAC	.	PASS	AN=88;AC=22,12,0;NS=22;DP=619;RCOUNT=847;END=17709;NVAR=13;SNVPOS=1,12,22,26,34,35,39,65,73,96,104,115,116;AFP=0.602,0.244,0.137,0.006	GT:GQ:PHQ:DP:RCOUNT:RCALLS:MEC:MECP:GPM:PHPM:MCI:AFP	0/0/1/2:28:28:30:43:383:2:0.005:0.998:0.998:0:0.5,0.25,0.25,0	0/0/0/1:3:3:22:28:280:2:0.007:0.48:0.48:0:0.62,0.25,0,0	0/0/1/2:60:60:31:41:401:1:0.002:1:1:0:0.5,0.25,0.25,0	0/0/1/2:6:6:14:26:176:0:0:0.752:0.761:0:0.438,0.251,0.252,0	0/0/1/2:17:18:25:31:320:0:0:0.982:0.983:0:0.5,0.25,0.246,0	0/0/1/2:60:60:

In [12]:
zcat assemble.AFP.vcf.gz | grep "locus012"

chr1	568849	locus012	TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA	TTGATAGGTTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATGGATTGGAAATGAGAGGTGTGAAGGTGCATACTCTCAACATAACACAGTTCTCCGAGTATCGAAAAGACGGTCACCCCTCAATTTACAGAAGGCAATGGGTGGCTTTACCAGAA,TTGATAGATTGGAAATGAGAGGGGTGAATGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGGTTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTGTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATGGATTGGAAATGAGAGGTGTGAAGGTGCATACTCTCAACATAACACAGTTCTCCGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAG

Note how the `AFP` field can substantially increase the size of the output VCF.
This is an even bigger issue for the `GP` and `GL` fields!

### Adjusting the threshold for reporting alleles

The `--haplotype-posterior-threshold` argument is used to adjust the number of haplotype alleles reported:

In [13]:
mchap assemble \
        --bam input/bam/*.bam \
        --targets input/bed/targets4.bed \
        --variants input/vcf/snvs.vcf.gz \
        --reference input/fasta/chr1.fa.gz \
        --ploidy 4 \
        --inbreeding 0.01 \
        --report AFP \
        --haplotype-posterior-threshold 0 \
        | bgzip > assemble.AFP.full.vcf.gz
tabix -p vcf assemble.AFP.full.vcf.gz

Notes on the `--haplotype-posterior-threshold` value:

- This must be a value between `0` and `1` (default is `0.2`)
- Haplotype-alleles will be reported if their probability of occurring  in any sample is greater than this value
- A low value increases the number of haplotype-alleles reported
- A high value decreases the number of haplotype-alleles reported
- Reporting more alleles results in a more detailed posterior distribution

(see the [documentation](https://github.com/PlantandFoodResearch/MCHap/blob/master/docs/assemble.rst#output-parameters) for more detail)

We can see how many "noise" haplotypes are reported when this value is zero, even at a well assembled locus:

In [14]:
zcat assemble.AFP.full.vcf.gz | grep "locus001"

chr1	17591	locus001	...	.	PASS	AN=88;AC=22,12,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

## Re-calling genotypes with `mchap call` (two-step approach)

*For more background information on* `mchap call` *see the online [documentation](https://github.com/PlantandFoodResearch/MCHap/blob/master/docs/call.rst).*

Recalling genotypes with `mchap call` has a few advantages:

- All genotypes will be complete
- Restriction of the prior distribution to a subset of alleles
- Can incorporate a prior on allele frequencies

This is most effective in a population of related samples

### Basic re-calling

A simple `mchap call` command:

In [15]:
mchap call \
        --bam input/bam/*.loci.bam \
        --haplotypes assemble.vcf.gz \
        --ploidy 4 \
        --inbreeding 0.01 \
        | bgzip > assemble.recall.vcf.gz
tabix -p vcf assemble.recall.vcf.gz

Notes:
- The `--haplotypes` argument replaces the `--variants` argument
- There is no `--reference` argument
- By default, it is assumed that all input alleles are equally frequent (i.e., a flat prior over frequencies)

Let's look at `locus001` again:

In [16]:
zcat assemble.recall.vcf.gz | grep "locus001"

chr1	17591	locus001	GTTATTGGACAGTGACGATGGAGTGATTGCTGGCGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGTACAC	ATTATTGGACAATGACGATGGGGTGGTTGCTGGCCCAGTCCGCCAGCACCACCACCACCAAGTCAACATGTCGGACATTTATGGGGTGGTGCCACGAAACCTGATCACAAATGGCGCAC,GTTATTGGACAGTGACGATGGAGTGATTGCTGGTGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGCACAC,ATTATTGGACAATGACGATGGGGTGGTTGCTGGCCCAGTCCGCCAGCACCACCACCACCAAGTCAACATGTCGGACATTTATGGGGTGGTGCCACGAAACCTGATCACAAATGGTGCAC	.	PASS	AN=88;AC=21,13,1;NS=22;DP=619;RCOUNT=847;END=17709;NVAR=13;SNVPOS=1,12,22,26,34,35,39,65,73,96,104,115,116	GT:GQ:PHQ:DP:RCOUNT:RCALLS:MEC:MECP:GPM:PHPM:MCI	0/0/1/2:60:60:30:43:383:2:0.005:1:1:0	0/0/0/1:8:8:22:28:280:2:0.007:0.824:0.824:0	0/0/1/2:60:60:31:41:401:1:0.002:1:1:0	0/0/1/2:17:25:14:26:176:0:0:0.979:0.997:0	0/0/1/2:60:60:25:31:320:0:0:1:1:0	0/0/1/2:60:60:31:46:408:0:0:1:1:0	0/0/0/1:60:60:22:33:291:0:0:1:1:0	0/0/1/2:13:13:21:26:272:1:0.004:0.952:0.954:0	0/0/2/3:3:3:38:50:497:1:0.0

Note that sample `progeny007` has been called with genotype `0/0/2/3` and that allele `3` is not found in any other sample.
This is a low-quality call with  `GQ = 3` and `GPM = 0.507`.
In the initial assembly, the genotype called for `progeny007` was `0/0/1/2` with `GQ = 3`, `GPM = 0.502`.
These posterior probabilities suggest that the genotype call of `progeny007` will be strongly influenced by the **prior distribution**.

### Specifying prior allele frequencies

In this example we use the population posterior allele frequencies from the initial assembly as prior allele frequencies for re-calling genotypes.
This assumes that the allele frequencies in a given individual should be proportional to the allele frequencies in the population as a whole.
This is a reasonable assumption for closely related samples (e.g., siblings).
This is slightly less realistic for the parents, however it is likely to be more realistic than a flat prior:

In [17]:
mchap call \
        --bam input/bam/*.loci.bam \
        --haplotypes assemble.AFP.vcf.gz \
        --ploidy 4 \
        --inbreeding 0.01 \
        --prior-frequencies AFP \
        | bgzip > assemble.AFP.prior.recall.vcf.gz
tabix -p vcf assemble.AFP.prior.recall.vcf.gz

Notes:

- The `--haplotype-frequencies` argument specifies a VCF INFO field to use as population prior frequencies

Looking at `locus001` again:

In [18]:
zcat assemble.AFP.prior.recall.vcf.gz | grep "locus001"

chr1	17591	locus001	GTTATTGGACAGTGACGATGGAGTGATTGCTGGCGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGTACAC	ATTATTGGACAATGACGATGGGGTGGTTGCTGGCCCAGTCCGCCAGCACCACCACCACCAAGTCAACATGTCGGACATTTATGGGGTGGTGCCACGAAACCTGATCACAAATGGCGCAC,GTTATTGGACAGTGACGATGGAGTGATTGCTGGTGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGCACAC,ATTATTGGACAATGACGATGGGGTGGTTGCTGGCCCAGTCCGCCAGCACCACCACCACCAAGTCAACATGTCGGACATTTATGGGGTGGTGCCACGAAACCTGATCACAAATGGTGCAC	.	PASS	AN=88;AC=22,12,0;NS=22;DP=619;RCOUNT=847;END=17709;NVAR=13;SNVPOS=1,12,22,26,34,35,39,65,73,96,104,115,116	GT:GQ:PHQ:DP:RCOUNT:RCALLS:MEC:MECP:GPM:PHPM:MCI	0/0/1/2:60:60:30:43:383:2:0.005:1:1:0	0/0/0/1:24:24:22:28:280:2:0.007:0.996:0.996:0	0/0/1/2:60:60:31:41:401:1:0.002:1:1:0	0/0/1/2:23:60:14:26:176:0:0:0.994:1:0	0/0/1/2:60:60:25:31:320:0:0:1:1:0	0/0/1/2:60:60:31:46:408:0:0:1:1:0	0/0/0/1:60:60:22:33:291:0:0:1:1:0	0/0/1/2:27:28:21:26:272:1:0.004:0.998:0.998:0	0/0/1/2:16:16:38:50:497:1:0.0

The `progeny007` now has genotype `0/0/1/2` with a posterior probability (`GPM`) of `0.976`.
In effect we have improved the genotype call of `progeny007` by incorporating information from the population in the form of a prior on allele frequencies.

Technically this approach is slightly biased because the prior was calculated using the initial assembly `AFP` of `progeny007` among other samples.
This means the reads observed from `progeny007` have been included in both the prior distribution and in the likelihood function.
However, this bias will quickly be diluted as the number of samples in the population increases.

### Two-step approach with a pooled assembly

The data we are working with is from a bi-parental 4x cross.
Based on this knowledge we can assume that there are, at most, 8 haplotypes present in the population.
Furthermore, assuming negligible selection, the allele frequencies of the population should approximate the combined dosage of the parents.
Therefore, we may be able to improve haplotype assembly by "pooling" all the sample data into a single pseudo-octoploid sample.
We can then re-call individual genotypes based on the alleles found in the pooled assembly.

We can pool all samples into a pseudo-sample using the `--sample-pool` argument:

In [19]:
mchap assemble \
        --bam input/bam/*.bam \
        --targets input/bed/targets4.bed \
        --variants input/vcf/snvs.vcf.gz \
        --reference input/fasta/chr1.fa.gz \
        --sample-pool POOL \
        --ploidy 8 \
        --report AFP \
        | bgzip > pooled.AFP.vcf.gz
tabix -p vcf pooled.AFP.vcf.gz

Notes:

- The `--sample-pool` parameter takes a name for our pooled "sample" in the output VCF file
- We set `--ploidy` to `8` to treat this pooled sample as an octoploid
- We have omitted  the `--inbreeding` argument, we could still include this but it would be interpreted as the *population* inbreeding coefficient

We can then re-call individual genotypes using the alleles found in the pooled sample: 

In [20]:
mchap call \
        --bam input/bam/*.loci.bam \
        --haplotypes pooled.AFP.vcf.gz \
        --ploidy 4 \
        --inbreeding 0.01 \
        --prior-frequencies AFP \
        | bgzip > pooled.AFP.prior.recall.vcf.gz
tabix -p vcf pooled.AFP.prior.recall.vcf.gz

The pooled two-step approach is most effective with large, unselected populations.
It can be particularly beneficial if your samples have low read depths resulting in poor quality individual assemblies.
However, assembling haplotypes at a higher ploidy can also be a cause of lower quality assemblies.
Therefore, the best improvement comes from having so many samples that their combined read depth is more than sufficient for assembly at the higher ploidy.
An additional advantage of a pooled assembly step with large populations is that it can be substantially faster than many individual assemblies.
The example dataset used here is too small to see a substantial improvement.

Lets lookl at `locus001` again:

In [21]:
zcat pooled.AFP.prior.recall.vcf.gz | grep "locus001"

chr1	17591	locus001	GTTATTGGACAGTGACGATGGAGTGATTGCTGGCGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGTACAC	ATTATTGGACAATGACGATGGGGTGGTTGCTGGCCCAGTCCGCCAGCACCACCACCACCAAGTCAACATGTCGGACATTTATGGGGTGGTGCCACGAAACCTGATCACAAATGGCGCAC,GTTATTGGACAGTGACGATGGAGTGATTGCTGGTGCAGGCCGCCAGCACCACCACCACCAAGTCGACATGTCCGACATTTATGGGGTGGTGCCACAAAACCTGCTCACAAATGGCACAC	.	PASS	AN=88;AC=22,12;NS=22;DP=619;RCOUNT=847;END=17709;NVAR=13;SNVPOS=1,12,22,26,34,35,39,65,73,96,104,115,116	GT:GQ:PHQ:DP:RCOUNT:RCALLS:MEC:MECP:GPM:PHPM:MCI	0/0/1/2:60:60:30:43:383:2:0.005:1:1:0	0/0/0/1:60:60:22:28:280:2:0.007:1:1:0	0/0/1/2:60:60:31:41:401:1:0.002:1:1:0	0/0/1/2:23:60:14:26:176:0:0:0.995:1:0	0/0/1/2:60:60:25:31:320:0:0:1:1:0	0/0/1/2:60:60:31:46:408:0:0:1:1:0	0/0/0/1:60:60:22:33:291:0:0:1:1:0	0/0/1/2:30:60:21:26:272:1:0.004:0.999:1:0	0/0/1/2:60:60:38:50:497:1:0.002:1:1:0	0/0/0/1:60:60:33:46:424:1:0.002:1:1:0	0/0/0/1:24:60:28:41:358:4:0.011:0.996:1:0	0/0/0/1:28:60:19:27:251:0:0:0.998:1:0	0/0/1/2

There are now only 3 alleles reported for `locus001` but the re-called genotypes seem to be the same.
This reflects that `locus001` seems to be a well-behaved locus for assembly.

Next we will look at `locus016` which we have not previously seen.
We start by checking the non-pooled two-step results:

In [22]:
zcat assemble.AFP.prior.recall.vcf.gz | grep "locus016"

chr1	684809	locus016	TCACATATGTTAAGCTATCGACCAATTGGCCAACAACAGGACCAAACAATGCAGTAGAGGCAGATTCGACCACACCGTAAACGGCAGGTAGGAGCAAGGAGTCTGGAAAAATATGAATC	TCACATATGTTAAGCTATCGACAAATTGGCCAACAACAGGACCAAACAATGCAGTAGAGGCAGATTCGACCACACCGTAAACGGCAGGTAGGAGTAAGGAGTCTGGGAAAATATGAATC,TCACATATGTTAAGCTATCAACCCATTGTCCAACAAGGGGCCCAAACAATGCCGTGGAAGCAGATTCCACCACACCATAAACAGCAGCTAAGAGTAATGAGTCTGGACAAATATCAATC,TCACATATGTTAAGCTATCGACCAATTGGCCAACAACAGGACCAAACAATGCAGTAGAGGCAGATTCGACCACACCGTAAACGGCAGGTAGGAGCAAGGAGTCTAGAAAAATATGAATC,TCACATATGTTAAGCTATCGACCAATTGGCCAACAACGGGACCAAACAATGCAGTAGAGTCAGATTCAACCACACCGTAAACGGCAGGTAGGAGTAAGGAGTCTGGAAAAATATGAATC,TCACATATGTTAAGCTATCGACAAATTGGCCAACAACAGGACCAAGCAATGCAGTAGAGGCAGATTCGACCACACCGTAAACGGCAGGTAGGAGTAAGGAGTCTGGAAAAATATGAATC,TCACATATGTTAAGCTATCAACCCATTGTCCAACAAGAGGCCCAAACAATGCCGTGGAAGCAGATTCCACCACACCATAAACAGCAGCTAAGAGTAATGAGTCTGGACAAATATCAATC,TCACATATGTTAAGCTATCAACCCATTGTCCAACAAGGGGCCCAAACAATGCCGTGGAAGCAGATTCCACCACACCATAAACAGCAGCTAGGAGTAAGGAGTCTGGACAAATATCAATC	.	PASS	AN=88;AC=22,

And now the pooled two-step results:

In [23]:
zcat pooled.AFP.prior.recall.vcf.gz | grep "locus016"

chr1	684809	locus016	TCACATATGTTAAGCTATCGACCAATTGGCCAACAACAGGACCAAACAATGCAGTAGAGGCAGATTCGACCACACCGTAAACGGCAGGTAGGAGCAAGGAGTCTGGAAAAATATGAATC	TCACATATGTTAAGCTATCGACAAATTGGCCAACAACAGGACCAAACAATGCAGTAGAGGCAGATTCGACCACACCGTAAACGGCAGGTAGGAGTAAGGAGTCTGGGAAAATATGAATC,TCACATATGTTAAGCTATCAACCCATTGTCCAACAAGGGGCCCAAACAATGCCGTGGAAGCAGATTCCACCACACCATAAACAGCAGCTAAGAGTAATGAGTCTGGACAAATATCAATC,TCACATATGTTAAGCTATCGACAAATTGGCCAACAACAGGACCAAGCAATGCAGTAGAGGCAGATTCGACCACACCGTAAACGGCAGGTAGGAGTAAGGAGTCTGGAAAAATATGAATC,TCACATATGTTAAGCTATCGACCAATTGGCCAACAACGGGACCAAACAATGCAGTAGAGTCAGATTCAACCACACCGTAAACGGCAGGTAGGAGTAAGGAGTCTGGAAAAATATGAATC,TCACATATGTTAAGCTATCGACCAATTGGCCAACAACAGGACCAAACAATGCAGTAGAGGCAGATTCGACCACACCGTAAACGGCAGGTAGGAGCAAGGAGTCTAGAAAAATATGAATC	.	PASS	AN=88;AC=22,22,13,10,11;NS=22;DP=3067;RCOUNT=3820;END=684927;NVAR=23;SNVPOS=20,23,24,29,37,38,41,46,53,56,59,60,68,77,83,88,91,95,98,105,107,108,115	GT:GQ:PHQ:DP:RCOUNT:RCALLS:MEC:MECP:GPM:PHPM:MCI	1/2/3/5:60:60:144:188:3309:7:0.002:1:1:0	0/1/2/4:60:60

At this locus there seems to be some minor improvements with a pooled assembly:

- alleles `6` and `7` are no longer reported
- genotype `3/4/5/6` is now `2/3/4/5` (more congruent with parents)


If we look at `locus012` again we see the pooled result is cleaner although still incongruent:

In [24]:
zcat assemble.AFP.prior.recall.vcf.gz | grep "locus012"

chr1	568849	locus012	TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA	TTGATAGGTTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATGGATTGGAAATGAGAGGTGTGAAGGTGCATACTCTCAACATAACACAGTTCTCCGAGTATCGAAAAGACGGTCACCCCTCAATTTACAGAAGGCAATGGGTGGCTTTACCAGAA,TTGATAGATTGGAAATGAGAGGGGTGAATGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGGTTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTGTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATGGATTGGAAATGAGAGGTGTGAAGGTGCATACTCTCAACATAACACAGTTCTCCGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAG

In [25]:
zcat pooled.AFP.prior.recall.vcf.gz | grep "locus012"

chr1	568849	locus012	TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA	TTGATAGGTTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATGGATTGGAAATGAGAGGTGTGAAGGTGCATACTCTCAACATAACACAGTTCTCCGAGTATCGAAAAGACGGTCACCCCTCAATTTACAGAAGGCAATGGGTGGCTTTACCAGAA,TTGATAGGTTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAGAGGGGTGAATGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTGTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGATTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAGCTCTCAGAGTATCGAAAAGACGGTCACCCCTCCATTTACAGAAGGCAATGGGTGGCTTTAACAGAA,TTGATAGGTTGGAAATGAGAGGGGTGAAGGTGCATATTCTCAACATAACACAACTTTCAGAGTATCGAAAAGACGGTCACCCCTCGATTTATAGAAGGCAATGGGTGGCTTTAACAGAA	.	PASS	AN=88;AC=23,

Allele `6` is present in many progeny but not found in either parent.